In [1]:
# 作者：MiloMusiala
# 链接：https://leetcode-cn.com/problems/evaluate-division/solution/pythonbing-cha-ji-fu-mo-ban-by-milomusia-kfsu/
# 来源：力扣（LeetCode）
# 著作权归作者所有。商业转载请联系作者获得授权，非商业转载请注明出处。

# [并查集](https://leetcode-cn.com/problems/number-of-provinces/solution/python-duo-tu-xiang-jie-bing-cha-ji-by-m-vjdr/)

## 基本概念

* 并查集是一种数据结构
* 并查集这三个字，一个字代表一个意思。
* 并（Union），代表合并
* 查（Find），代表查找
* 集（Set），代表这是一个以字典为基础的数据结构，它的基本功能是合并集合中的元素，查找集合中的元素
* 并查集的典型应用是有关连通分量的问题
* 并查集解决单个问题（添加，合并，查找）的时间复杂度都是O(1)。因此，并查集可以应用到在线算法中

## 并查集的实现

数据结构：并查集跟树有些类似，只不过她跟树是相反的。在树这个数据结构里面，每个节点会记录它的子节点。在并查集里，每个节点会记录它的父节点。

<img src="https://pic.leetcode-cn.com/1609980000-ofFjdW-%E5%B9%BB%E7%81%AF%E7%89%871.JPG" width=50%>

可以看到，如果节点是相互连通的（从一个节点可以到达另一个节点），那么他们在同一棵树里，或者说在同一个集合里，或者说他们的祖先是相同的。

## 初始化
当把一个新节点添加到并查集中，它的父节点应该为空

<img src="https://pic.leetcode-cn.com/1609980044-MZZZkZ-%E5%B9%BB%E7%81%AF%E7%89%872.JPG" width=50%>

## 合并两个节点
如果发现两个节点是连通的，那么就要把他们合并，也就是他们的祖先是相同的。这里究竟把谁当做父节点一般是没有区别的。

<img src="https://pic.leetcode-cn.com/1609980079-JmfIbX-%E5%B9%BB%E7%81%AF%E7%89%874.JPG" width=50%>

## 两节点是否连通
我们判断两个节点是否处于同一个连通分量的时候，就需要判断它们的祖先是否相同

## 查找祖先
查找祖先的方法是：如果节点的父节点不为空，那就不断迭代。

<img src="https://pic.leetcode-cn.com/1609980128-GpUyoj-%E5%B9%BB%E7%81%AF%E7%89%877.JPG" width=50%>

这里有一个优化的点：如果我们树很深，比如说退化成链表，那么每次查询的效率都会非常低。所以我们要做一下路径压缩。也就是把树的深度固定为二。

<img src="https://pic.leetcode-cn.com/1609980188-XcJBuX-%E5%B9%BB%E7%81%AF%E7%89%8711.JPG" width=50%>

这么做可行的原因是，并查集只是记录了节点之间的连通关系，而节点相互连通只需要有一个相同的祖先就可以了。

路径压缩可以用递归，也可以迭代。这里用迭代的方法。


In [3]:
class UnionFind:
    def __init__(self):
        """
        记录每个节点的父节点,记录每个节点到根节点的权重
        """
        self.father = {}
        self.value = {}
    
    def find(self,x): # 边查边更新
        """
        查找根节点 路径压缩 更新权重
        """
        root = x    
        base = 1 # 节点更新权重的时候要放大的倍数
        while self.father[root] != None: # 查找根节点及与根节点距离
            root = self.father[root]
            base *= self.value[root]     
        while x != root: # 更新根节点及与根节点距离、压缩路径
            original_father = self.father[x]
            self.value[x] *= base
            base /= self.value[original_father]
            self.father[x] = root
            x = original_father
        return root

    def merge(self,x,y,val):
        """
        合并两个节点
        """
        root_x,root_y = self.find(x),self.find(y) 
        if root_x != root_y:
            self.father[root_x] = root_y
            self.value[root_x] = self.value[y] * val / self.value[x]

    def is_connected(self,x,y):
        """
        两节点是否相连
        """
        return x in self.value and y in self.value and self.find(x) == self.find(y)
    
    def add(self,x):
        """
        添加新节点，初始化权重为1.0
        """
        if x not in self.father:
            self.father[x] = None
            self.value[x] = 1.0

# 399. 除法求值

给你一个变量对数组equations和一个实数值数组values作为已知条件，其 equations\[i] =\[Ai,Bi]和values\[i]共同表示等式 Ai/Bi = values\[i]。每个Ai或Bi是一个表示单个变量的字符串。

另有一些以数组queries表示的问题，其中queries\[j]=\[Cj,Dj]表示第j个问题，请你根据已知条件找出Cj/Dj =?的结果作为答案。

返回所有问题的答案 。如果存在某个无法确定的答案，则用 -1.0 替代这个答案。如果问题中出现了给定的已知条件中没有出现的字符串，也需要用 -1.0 替代这个答案。

注意：输入总是有效的。你可以假设除法运算中不会出现除数为 0 的情况，且不存在任何矛盾的结果。


In [13]:
class Solution:
    def calcEquation(self, equations, values, queries):
        '''
            equations: List[List[str]]
            values: List[float], 
            queries: List[List[str]]) 
            out: List[float]
        '''
        uf = UnionFind()
        for (a,b),val in zip(equations,values):
            uf.add(a)
            uf.add(b)
            uf.merge(a,b,val)
        
        res = [-1.0] * len(queries)
        print("单纯的并集：", uf.father, uf.value)
        '''
            在以上的步骤当中，主要是将两两之间的节点联系起来，
            如果添加的节点当中有以前的子节点，那么更新该节点
            到根节点之间的节点的根节点，而不更新该节点的子节
            点的根节点。
            在下面的步骤中，判断两个节点是否有相同的根节点，
            判断过程中更新该节点与根节点之间节点的根节点。
        '''
        for i,(a,b) in enumerate(queries):
            if uf.is_connected(a,b):
                res[i] = uf.value[a] / uf.value[b]
        print("查找更新之后：", uf.father, uf.value)
        return res

equations = [["a", "b"], ["b", "c"]]
values = [2.0, 3.0]
queries = [["a", "c"], ["b", "a"], ["a", "e"], ["a", "a"], ["x", "x"]]

sol = Solution()
results = sol.calcEquation(equations, values, queries)
print("results:",results)

# print("**************************练习2***********************************")
# equations = [["a","b"],["b","c"],["bc","cd"]]
# values = [1.5,2.5,5.0]
# queries = [["a","c"],["c","b"],["bc","cd"],["cd","bc"],["cd","b"]]
# sol = Solution()
# results = sol.calcEquation(equations, values, queries)
# print("results:",results)


单纯的并集： {'a': 'b', 'b': 'c', 'c': None} {'a': 2.0, 'b': 3.0, 'c': 1.0}
查找更新之后： {'a': 'c', 'b': 'c', 'c': None} {'a': 6.0, 'b': 3.0, 'c': 1.0}
results: [6.0, 0.5, -1.0, 1.0, -1.0]


In [10]:
1

1